In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
import numpy as np 
import pandas as pd 
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression # our model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [5]:
train

In [6]:
train=train.drop(['keyword','location'],axis=1)
y=train.pop('target')

In [7]:
def preprocessing(text):
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    text=text.lower()
    text = re.sub(r'\d+', '', text)
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    text = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    def remove_stopwords(text):
        text = [i for i in text if not i in stop_words]
    lemmatizer = WordNetLemmatizer()
    def lemmatize(text):
        text = [lemmatizer.lemmatize(token) for token in text]
    text = ' '.join(text)
    return text

In [8]:
text_train = []
for text_data in train['text']:
    text_data = preprocessing(text_data)
    text_train.append(text_data)
train['new_train'] = text_train
pre_test = []
for text_dat in test['text']:
    text_dat = preprocessing(text_dat)
    pre_test.append(text_dat)
test['new_test'] = pre_test

In [10]:
train_text_data = list(train['new_train'])
test_text_data = list(test['new_test'])
corpus = train_text_data + test_text_data

In [13]:
tf=TfidfVectorizer()
fitted_vectorizer = tf.fit(corpus)

In [14]:
train_transform = fitted_vectorizer.transform(train['new_train'])
test_transform = fitted_vectorizer.transform(test['new_test'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(train_transform, y)
log_reg = LogisticRegression()
model=log_reg.fit(X_train, y_train)

In [17]:
y_pred = model.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

In [19]:
random_clf = RandomForestClassifier(n_estimators=250)
model2=random_clf.fit(X_train, y_train)

In [21]:
y_pred2 = model2.predict(X_test)
accuracy_score(y_test, y_pred2)

In [22]:
y_pred